In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [22]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2, 2)
        x = x.view(-1, 32 * 56 * 56)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [23]:
def load_data():
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    data_transform = datasets.ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/Pneumonia/X-Ray', transform=transform)

    train_size = int(0.75 * len(data_transform))
    val_size = int(0.15 * len(data_transform))
    test_size = len(data_transform) - train_size - val_size

    train_data, val_data, test_data = torch.utils.data.random_split(data_transform, [train_size, val_size, test_size])

    extra_val_data, train_data = torch.utils.data.random_split(train_data, [int(0.1 * len(train_data)), len(train_data) - int(0.1 * len(train_data))])

    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=32)
    test_loader = DataLoader(test_data, batch_size=32)

    return train_loader, val_loader, test_loader


In [24]:
def train_evaluate_cnn(model, train_loader, val_loader, test_loader, epochs=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    train_accuracy_list = []
    val_accuracy_list = []
    test_accuracy_list = []

    for epoch in range(epochs):
        model.train()
        total_train, correct_train = 0, 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        train_accuracy = 100 * correct_train / total_train
        train_accuracy_list.append(train_accuracy)

        model.eval()
        total_val, correct_val = 0, 0
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

        val_accuracy = 100 * correct_val / total_val
        val_accuracy_list.append(val_accuracy)

        model.eval()
        total_test, correct_test = 0, 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total_test += labels.size(0)
                correct_test += (predicted == labels).sum().item()

        test_accuracy = 100 * correct_test / total_test
        test_accuracy_list.append(test_accuracy)

    return train_accuracy_list, val_accuracy_list, test_accuracy_list

In [ ]:
train_loader, val_loader, test_loader = load_data()
cnn_model = CNN()
train_acc_list, val_acc_list, test_acc_list = train_evaluate_cnn(cnn_model, train_loader, val_loader, test_loader)

In [ ]:
print(f"Train Accuracies: {train_acc_list}")
print(f"Validation Accuracies: {val_acc_list}")
print(f"Test Accuracies: {test_acc_list}")